In [1]:
import re
import matplotlib.pyplot as plt
import numpy as np
import cv2
from glob import glob
import os
import nibabel as nib
import torch
import torch.nn as nn
from skimage.transform import resize
import torch.nn.functional as F

In [2]:
root = "../data_for_training/Synapse/RawData/Training"

root_images = os.path.join(root, "img")
root_labels = os.path.join(root, "label")

print(root_images, root_labels)

../data_for_training/Synapse/RawData/Training\img ../data_for_training/Synapse/RawData/Training\label


In [3]:
def get_all_files(folder_path, formatted_extension): 
    paths = sorted(glob(os.path.join(folder_path, formatted_extension)))

    return paths

In [4]:
images = get_all_files(root_images, "*.nii.gz")
labels = get_all_files(root_labels, "*.nii.gz")

print(len(images), len(labels))

30 30


In [5]:
for x, y in zip (images, labels): 
    print(x, y) 

../data_for_training/Synapse/RawData/Training\img\img0001.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0001.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0002.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0002.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0003.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0003.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0004.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0004.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0005.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0005.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0006.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0006.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0007.nii.gz ../data_for_training/Synapse/RawData/Training\label\label0007.nii.gz
../data_for_training/Synapse/RawData/Training\img\img0008.nii.

In [15]:
a = nib.load("../data_for_training/Synapse/RawData/Training\img\img0039.nii.gz").get_fdata()

print(np.min(a), np.max(a), np.unique(a), a.shape) 

slice = a[:, :, 30]

print(np.unique(slice)) 

-1024.0 1541.0 [-1024. -1023. -1022. ...  1513.  1525.  1541.] (512, 512, 90)
[-1024. -1023. -1022. ...  1017.  1039.  1073.]


In [12]:
slice = torch.from_numpy(slice) 

slice.shape

torch.Size([512, 512])

In [13]:
slice = slice.to(torch.int64)

slice = F.one_hot(slice, num_classes=14)

slice = slice.numpy()


b = resize(slice, (256, 256, 14), preserve_range=True, anti_aliasing=True)
encoded_mask_reshape = np.argmax(b, axis=-1) 

print(np.unique(encoded_mask_reshape))

[ 0  2  3  4  6  7  8  9 11]
